In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd

from keras.datasets import mnist
import tensorflow.keras as kb
from tensorflow.keras import backend
import tensorflow as tf
from sklearn.preprocessing import LabelBinarizer


from plotnine import *

from sklearn.metrics import mean_squared_error, mean_absolute_error, roc_auc_score

from sklearn.linear_model import LinearRegression # Linear Regression Model
from sklearn.preprocessing import StandardScaler #Z-score variables

from sklearn.model_selection import train_test_split # simple TT split cv
from sklearn.model_selection import KFold # k-fold cv
from sklearn.model_selection import LeaveOneOut #LOO cv


# Optimizers

In the lecture we talked about different methods for optimizing our loss function

- **Gradient Descent**
- Gradient Descent with **Momentum**
- **AdaGrad**
- **RMSP**
- **Adam**

All of these are (or are based off of) the basic updating rule from Gradient Desent:

$$ w_t = w_{t-1} - \alpha * g_t $$

Which says that the new weights ($w_t$) are the old weights ($w_{t-1}$) minus some adjustment which is the product of the learning rate ($\alpha$) and the gradient ($g_t$). 


**Momentum** affects the gradient part of the update rule. Rather than updating based on just the current gradient, we update based on the *moving average* of past gradients. This allows us to "build up" speed as we "roll" down the gradient, but also smooths out any osscilating steps that we might take. 

$$ w_t = w_{t-1} - \alpha * m_t $$
$$ m_t = \beta * m_{t-1} + (1 - \beta) * g_t$$

**AdaGrad** and **RMSP** both affect the learning rate part of the update rule. Both optimizers allow us to

1. use different learning rates for different weights/parameters
2. **ada**pt the learning rate throughout the process

AdaGrad:
$$ w_t = w_{t-1} - \frac{\alpha}{\sqrt{\epsilon + \sum g_t^2}} * g_t $$

RMSP:
$$ w_t = w_{t-1} - \frac{\alpha}{\sqrt{\epsilon + \nu_t}} * g_t $$
$$ \nu_t = \beta * \nu_{t-1} + (1 - \beta) * g_t^2$$
AdaGrad updates the learning rate based on the sum of the squared past gradients, whereas RMSP updates the learning rate based on the moving average of the past squared gradients. 

**Adam** combines the changes Momentum makes to the gradient part of the update rule, and the changes RMSP makes to the learning rate. It also unbiases the momentum and learning rate parameters so that they are not as strongly affected by the fact that we initialize the past gradients to be 0. 

$$ w_t = w_{t-1} - \frac{\alpha}{\sqrt{\epsilon + \hat{\nu_t}}} * \hat{m_t} $$
$$ \nu_t = \beta_2 * \nu_{t-1} + (1 - \beta_2) * g_t^2$$
$$m_t = \beta_1 * m_{t-1} + (1 - \beta_1) * g_t$$
$$ \hat{\nu_t} = \frac{\nu_t}{1 - \beta_2^t},\; \hat{m_t} = \frac{m_t}{1 - \beta_1^t}$$

## Using Different Optimizers in Keras/Tensorflow


## Previously

In a past lecture, we looked at a python implementation of Gradient Descent for a simple linear regression model using a Sum of Square Errors loss function. The function below, `stepGradient()` takes in four arguments:
- `b0_current`: the current value for the `b0` intercept parameter
- `b1_current`: the current value for the `b1` slope parameter
- `point` a DataFrame of the points we're using to claculate the gradient
- `learningRate` a constant value representing the learning rate (how big of a step we should take at each step)


### Question
<img src="https://drive.google.com/uc?export=view&id=1ghyQPx1N8dmU3MV4TrANvqNhGwnLni72" alt="Q" width = "200"/>

Take a moment to study this function and map it to the process that we learned about for gradient descent. Call this function using `b0_current` = 0, `b1_current` = 0, `points` = `df`, and `learningRate` = 0.1

In [ ]:
def stepGradient(b0_current, b1_current, points, learningRate):
    # initialize gradient to 0
    b0_gradient = 0
    b1_gradient = 0

    # for each data point, calculate gradient and add 
    for i in range(0, len(points)):
        b0_gradient += -2 * (points.iloc[i].y - ((b1_current*points.iloc[i].x) + b0_current))
        b1_gradient += -2 * points.iloc[i].x * (points.iloc[i].y - ((b1_current * points.iloc[i].x) + b0_current))

    b0_gradient = b0_gradient/points.shape[0]
    b1_gradient = b1_gradient/points.shape[0]
    # update parameter values
    new_b0 = b0_current - (learningRate * b0_gradient)
    new_b1 = b1_current - (learningRate * b1_gradient)
    return [np.round(new_b0,5), np.round(new_b1,5)]

#based on https://spin.atomicobject.com/2014/06/24/gradient-descent-linear-regression/

In [ ]:
# create data frame 
np.random.seed(1234)
x = np.random.normal(loc = 0, scale = 1, size = 100)
y = 1.67 + 0.67*x + np.random.normal(loc = 0, scale = 0.2, size = 100)
df = pd.DataFrame({"x": x, "y": y})
df

In [ ]:
# GD
a = 0
b = 0

# run 500 updates
for i in range(0,500):
  a,b = stepGradient(a,b, df, 0.1)

  # every 10 updates, print the current parameter values
  if i%10 == 0:
    print(a,b)

## Momentum
Now that we've learned about momentum, let's modify our `stepGradient()` function to incorporate momentum. 

Remember momentum is calculated based on a decaying sum of the past gradients

$$w_t = w_{t-1} - \alpha * m_t$$
$$m_t = \beta * m_{t-1} + (1-\beta)* g_t$$

This new `stepMomentum()` function will need to take in **three** additional arguments:

- `b0_mt`: $m_t$; the sum of the gradients for `b0` from the previous step
- `b1_mt`: $m_t$; the sum of the gradients for `b1` from the previous step
- `beta`: moving average parameter that controlls how much of the previous gradient is remembered (set to a default value of `0.9`)


### Question
<img src="https://drive.google.com/uc?export=view&id=1ghyQPx1N8dmU3MV4TrANvqNhGwnLni72" alt="Q" width = "200"/>

I have added these arguments to the function definition for you. Take a moment to look at the lecture slides and understand the difference between Gradient Descent and Gradient Descent *with* momentum. 

Then, modify the function below (which contains the same code as the `stepGradient()` function) to implement Gradient Descent *with* momentum.

In [ ]:
### YOUR CODE HERE ###
# Change this function so it does GD with momentum, right now it just does GD

def stepMomentum(b0_current, b1_current, points, learningRate):
    # initialize gradient to 0
    b0_gradient = 0
    b1_gradient = 0

    # for each data point, calculate gradient and add 
    for i in range(0, len(points)):
        b0_gradient += -2 * (points.iloc[i].y - ((b1_current*points.iloc[i].x) + b0_current))
        b1_gradient += -2 * points.iloc[i].x * (points.iloc[i].y - ((b1_current * points.iloc[i].x) + b0_current))

    b0_gradient = b0_gradient/points.shape[0]
    b1_gradient = b1_gradient/points.shape[0]
    # update parameter values
    new_b0 = b0_current - (learningRate * b0_gradient)
    new_b1 = b1_current - (learningRate * b1_gradient)
    return [np.round(new_b0,5), np.round(new_b1,5)]

#based on https://spin.atomicobject.com/2014/06/24/gradient-descent-linear-regression/

## Try it Out

Once you've created and tested your `stepMomentum()` function, try it out with this dataset, `df` and compare it to the output of the `stepGradient()` function on the same data.

Initialize `b0_mt` and `b1_mt` both to 0.

### Question
<img src="https://drive.google.com/uc?export=view&id=1ghyQPx1N8dmU3MV4TrANvqNhGwnLni72" alt="Q" width = "200"/>

What's different about the updates you got?

In [ ]:
### YOUR CODE HERE ###

### Question
<img src="https://drive.google.com/uc?export=view&id=1ghyQPx1N8dmU3MV4TrANvqNhGwnLni72" alt="Q" width = "200"/>

What do you think the potential downside of initializing `b0_mt` and `b1_mt` to `0` is?

## AdaGrad

We also learned about an upgraded version of Gradient Descent called **AdaGrad**. AdaGrad updates the learning rate for each parameter separately by scaling a baseline learning rate, $\alpha$ by the square root of the sum of the previous gradients.

Similarly to what we did with momentum, update the code from the `stepGradient()` function below to implement AdaGrad. 

The AdaGrad will need two extra arguments:
- `b0_squared_gradient_sum`: the sum of the previous squared gradients for `b0`
- `b1_squared_gradient_sum`: the sum of the previous squared gradients for `b1`

(Set $\epsilon$ to be 0.0001)

In [ ]:
def stepAda(b0_current, b1_current, points, learningRate,b0_squared_gradient_sum, b1_squared_gradient_sum ):
    pass
    # COPY THE GRADIENT DESCENT FUNCTION AND ALTER IT

## RMSProp

We also learned about another algorithm, RMSProp which:

- scales the learning rate by the moving average of the squared past gradients
- tailors the learning rate for each parameter separately



In [ ]:
def stepRMSP(b0_current, b1_current, points, learningRate,b0_squared_gradient_sum, b1_squared_gradient_sum, beta ):
    # initialize gradient to 0
    b0_gradient = 0
    b1_gradient = 0

    # for each data point, calculate gradient and add 
    for i in range(0, len(points)):
        b0_gradient += (1/10000) * -2 * (points.iloc[i].y - ((b1_current*points.iloc[i].x) + b0_current))
        b1_gradient += (1/10000) * -2 * points.iloc[i].x * (points.iloc[i].y - ((b1_current * points.iloc[i].x) + b0_current))
    b0_gradient = b0_gradient/points.shape[0]
    b1_gradient = b1_gradient/points.shape[0]
    
    # new learning rates
    b0_squared_gradient_sum = (beta *b0_squared_gradient_sum) + ((1-beta) * b0_gradient**2)
    b1_squared_gradient_sum = (beta *b1_squared_gradient_sum) + ((1-beta) * b1_gradient**2)

    b0_learningRate = learningRate/(np.sqrt(0.0001 + b0_squared_gradient_sum))
    b1_learningRate = learningRate/(np.sqrt(0.0001 + b1_squared_gradient_sum))

    
    # update parameter values
    new_b0 = b0_current - (b0_learningRate * b0_gradient)
    new_b1 = b1_current - (b1_learningRate * b1_gradient)
    return [np.round(new_b0,5), np.round(new_b1,5), b0_squared_gradient_sum, b1_squared_gradient_sum]
#based on https://spin.atomicobject.com/2014/06/24/gradient-descent-linear-regression/

## Adam

Finally, we learned about Adam, which combines Momentum and RMSP.

I won't make you figure this one out on your own, but the code is below!

In [ ]:
def stepAdam(b0_current, b1_current, points, learningRate,
             b0_squared_gradient_sum, b1_squared_gradient_sum,
             b0_mt, b1_mt, beta1, beta2,t ):
    # initialize gradient to 0
    b0_gradient = 0
    b1_gradient = 0

    # for each data point, calculate gradient and add 
    for i in range(0, len(points)):
        b0_gradient += (1/10000) * -2 * (points.iloc[i].y - ((b1_current*points.iloc[i].x) + b0_current))
        b1_gradient += (1/10000) * -2 * points.iloc[i].x * (points.iloc[i].y - ((b1_current * points.iloc[i].x) + b0_current))
    
    b0_gradient = b0_gradient/points.shape[0]
    b1_gradient = b1_gradient/points.shape[0]

    # calculate mts
    b0_mt = (beta1*b0_mt) + ((1-beta1)*b0_gradient)
    b1_mt = (beta1*b1_mt) + ((1-beta1)*b1_gradient)

    # unbias these estimates
    b0_mt_unbiased = b0_mt/(1-beta1**t)
    b1_mt_unbiased = b1_mt/(1-beta1**t)

    # new learning rates
    b0_squared_gradient_sum = (beta2 *b0_squared_gradient_sum) + ((1-beta2) * b0_gradient**2)
    b1_squared_gradient_sum = (beta2 *b1_squared_gradient_sum) + ((1-beta2) * b1_gradient**2)

    # unbias these estimates
    b0_squared_gradient_sum_unbiased = b0_squared_gradient_sum/(1-beta2**t)
    b1_squared_gradient_sum_unbiased = b1_squared_gradient_sum/(1-beta2**t)

    b0_learningRate = learningRate/(np.sqrt(0.0001 + b0_squared_gradient_sum_unbiased))
    b1_learningRate = learningRate/(np.sqrt(0.0001 + b1_squared_gradient_sum_unbiased))

    
    # update parameter values
    new_b0 = b0_current - (b0_learningRate * b0_mt_unbiased)
    new_b1 = b1_current - (b1_learningRate * b1_mt_unbiased)
    return [np.round(new_b0,5), np.round(new_b1,5), b0_squared_gradient_sum, b1_squared_gradient_sum, b0_mt, b1_mt]
#based on https://spin.atomicobject.com/2014/06/24/gradient-descent-linear-regression/

## Linear Regression (Comparison)

Just for comparison, since this is a simple model, let's see what Linear Regression comes up with for this model!

### Question
<img src="https://drive.google.com/uc?export=view&id=1ghyQPx1N8dmU3MV4TrANvqNhGwnLni72" alt="Q" width = "200"/>

How close did our various methods of Gradient Descent get to Linear Regression's parameter estimates?

In [ ]:
from sklearn.linear_model import LinearRegression 
lr = LinearRegression()

lr.fit(df[["x"]], df["y"])
print(lr.intercept_, lr.coef_[0])